# install packages

In [1]:
!pip install pyyaml==5.3.1

In [2]:
!pip install gen3

In [3]:
!pip install pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.1 MB/s eta 0:00:00


In [4]:
import gen3
import os
import time

In [5]:
# test pulling MIDRC GUIDs -- this works
# os.system("gen3 --auth midrc_credentials.json --endpoint data.midrc.org drs-pull object dg.MD1R/ea669b5e-ae51-40ba-b375-ed23a9cd1855")
          

# write nextflow config

In [7]:
%%writefile nextflow.config

process {
    withLabel: download_data {
        executor = 'local'
    }
    withLabel: publish_data {
        executor = 'local'
    }
    withLabel: dcm2png {
        executor = 'local'
    }
}

workDir = 'sdk_data'

Overwriting nextflow.config


# Pull some midrc guids to test

In [8]:
%%writefile download_data.py

import gen3
import os

output_dir = 'data'

guids = ['dg.MD1R/ea669b5e-ae51-40ba-b375-ed23a9cd1855',
         'dg.MD1R/a745ed98-0cb9-4537-826b-13b2e354e8bb',
         'dg.MD1R/e604979a-c71b-4ec6-b8a0-959837b86384',
         'dg.MD1R/b5cee98d-46ff-4438-aa00-90727a383340',
         'dg.MD1R/8a5a5579-7925-432d-a614-3ed208f1c182',
         'dg.MD1R/33034812-47f3-4c0e-b60b-fa7a2a04ecda',
         'dg.MD1R/5ca987c5-c660-4785-a67d-a3424cc8ec6e',
         'dg.MD1R/44148117-1858-49ef-b30f-d239abfaff80',
         'dg.MD1R/9ea205e8-a774-4318-a323-95eadda9bc5c',
         'dg.MD1R/09ece36f-a0fa-48e8-8fc2-62110eaae570']

midrc_creds_file = "/home/jovyan/pd/midrc_credentials.json"
endpoint = 'data.midrc.org'

for guid in guids:
    print('pulling guid {}'.format(guid))
    os.system("gen3 --auth {} --endpoint {} drs-pull object {}".format(midrc_creds_file, endpoint, guid))

Overwriting download_data.py


In [13]:
%%writefile main.nf
#!/usr/bin/env nextflow

process DownloadData {
    
    label 'download_data'
    
    tag 'initial_task'
    
    output:
    stdout emit: download_data_log
    path('**/*.dcm'), emit: dicom_image
    
    script:
    """
    python3 ${baseDir}/download_data.py
    """
}

process PublishData {
    
    label 'publish_data'
    
    publishDir "${baseDir}/dicom_files", mode: 'copy'
    
    input:
    path(dicom_image) 
    
    output:
    stdout emit: convert_to_png_log
    
    script:
    """
    echo $dicom_image
    """
}

// Define the entry workflow (initial workflow for Nextflow to run)
workflow {
    DownloadData()
    PublishData(DownloadData.out.dicom_image)
}


Overwriting main.nf


In [14]:
!nextflow run main.nf -dsl2

N E X T F L O W  ~  version 21.10.6
Launching `main.nf` [distracted_lalande] - revision: 9cf41d1605
executor >  local (1)
[cb/b8cf2a] process > DownloadData (initial_task) [  0%] 0 of 1
[-        ] process > PublishData                 -

executor >  local (1)
[cb/b8cf2a] process > DownloadData (initial_task) [  0%] 0 of 1
[-        ] process > PublishData                 -

executor >  local (2)
[cb/b8cf2a] process > DownloadData (initial_task) [100%] 1 of 1 ✔
[b7/a9dbb4] process > PublishData                 [  0%] 0 of 1

executor >  local (2)
[cb/b8cf2a] process > DownloadData (initial_task) [100%] 1 of 1 ✔
[b7/a9dbb4] process > PublishData                 [100%] 1 of 1 ✔

